# Imports

In [63]:
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer, PolynomialFeatures
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sqlalchemy import create_engine

import credentials

# Data loading

In [3]:
db_con = create_engine(credentials.DB_URL)

train_df = pd.read_sql('SELECT * FROM cleaned.feat_reduced_train_set', db_con)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2620 entries, 0 to 2619
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ph                           2620 non-null   float64
 1   ph Sulfate                   2620 non-null   float64
 2   ph Chloramines               2620 non-null   float64
 3   ph Hardness                  2620 non-null   float64
 4   Chloramines Trihalomethanes  2620 non-null   float64
 5   Hardness Sulfate             2620 non-null   float64
 6   Chloramines                  2620 non-null   float64
 7   Chloramines Solids_sqrt      2620 non-null   float64
 8   Hardness Solids_sqrt         2620 non-null   float64
 9   Turbidity Solids_sqrt        2620 non-null   float64
 10  Conductivity Solids_sqrt     2620 non-null   float64
 11  Sulfate Conductivity         2620 non-null   float64
 12  Sulfate                      2620 non-null   float64
 13  Hardness Turbidity

This contains all the features I selected last week using random forest feature importance and forward sequential selection of which many are engineered features.

In [4]:
train_df.describe()

,ph,ph Sulfate,ph Chloramines,ph Hardness,Chloramines Trihalomethanes,Hardness Sulfate,Chloramines,Chloramines Solids_sqrt,Hardness Solids_sqrt,Turbidity Solids_sqrt,...,Sulfate,Hardness Turbidity,Hardness,Hardness Conductivity,Chloramines Sulfate,Solids_sqrt,ph Conductivity,Hardness Chloramines,ph Solids_sqrt,not_potible
count,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,...,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000,2620.000000
mean,7.065416,2363.263735,50.348068,1385.941706,471.724045,65329.027761,7.133225,1033.437014,28404.109287,578.063682,...,334.494997,776.951008,195.723754,83328.213207,2387.439227,145.356431,3012.550009,1393.763826,1024.414717,0.605725
std,1.459029,553.811075,15.355133,379.018210,158.372356,12290.030865,1.599750,308.090686,7466.704099,167.130950,...,35.942568,198.446514,32.467513,21059.080933,597.733539,29.905886,861.667847,387.167343,296.075932,0.488788
min,0.227499,64.540066,0.787714,34.700455,5.469344,17793.987434,0.352000,45.047958,3855.129399,62.179151,...,129.000000,194.799642,47.432000,23727.665988,83.933191,17.914871,100.788730,81.698615,44.943933,0.000000
25%,6.280114,2023.804618,40.383627,1140.064277,361.513236,58167.079922,6.136351,823.955777,23239.340324,456.445684,...,317.943278,638.838723,176.211415,68427.489474,2020.577445,124.937104,2436.892474,1150.383578,823.896803,0.000000
50%,7.065416,2359.648234,49.481758,1374.743151,461.815600,64972.100383,7.122437,1010.515088,27920.943395,571.015251,...,334.494997,768.150769,196.637469,82092.550718,2365.643771,144.627208,2938.352653,1364.135329,1007.588986,1.000000
75%,7.837675,2677.930438,59.340460,1601.474142,570.705078,72513.543142,8.137069,1206.408464,32922.682628,684.487693,...,350.538503,906.403819,215.942102,96225.248679,2759.009172,165.397445,3515.495079,1599.627018,1203.155194,1.000000
max,14.000000,5335.367960,132.465573,3299.484406,1188.316898,115330.178785,13.127000,2772.783700,66667.038283,1279.926734,...,481.030642,1622.073945,317.338124,196608.096768,5038.646434,247.441298,6590.941376,2898.196893,2346.135032,1.000000


In [5]:
train_df

,ph,ph Sulfate,ph Chloramines,ph Hardness,Chloramines Trihalomethanes,Hardness Sulfate,Chloramines,Chloramines Solids_sqrt,Hardness Solids_sqrt,Turbidity Solids_sqrt,...,Sulfate,Hardness Turbidity,Hardness,Hardness Conductivity,Chloramines Sulfate,Solids_sqrt,ph Conductivity,Hardness Chloramines,ph Solids_sqrt,not_potible
0,8.846282,2805.603139,53.589605,1666.772221,364.996086,59755.920045,6.057868,806.742172,25091.721339,389.534947,...,317.150545,551.120953,188.415001,79020.416411,1921.256023,133.172631,3710.091455,1141.393137,1178.082642,1.0
1,5.166226,1728.076740,44.621199,701.119717,656.998674,45395.040580,8.637098,948.509458,14903.648787,506.076763,...,334.494997,625.405007,135.712166,49157.288652,2889.066024,109.818075,1871.296204,1172.159259,567.344989,1.0
2,8.195765,2681.604414,51.595662,1758.136000,458.033603,70188.853114,6.295405,641.685002,21865.588131,328.080152,...,327.193898,690.469811,214.517610,86491.336668,2059.818108,101.929105,3304.449922,1350.475242,835.387032,0.0
3,6.809645,2277.792022,59.121221,1223.826048,631.335777,60115.280499,8.681983,1624.276941,33622.993448,595.644849,...,334.494997,572.191652,179.719521,64866.670463,2904.080034,187.085929,2457.824088,1560.321906,1273.988670,0.0
4,7.065416,2211.155138,71.281368,1335.371317,630.902040,59148.777172,10.088771,1454.483453,27248.010599,742.402015,...,312.954695,973.269115,189.001086,79292.406790,3157.328397,144.168540,2964.183219,1906.788767,1018.610722,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615,7.965337,2806.317228,56.602033,1207.082261,371.422161,53390.659717,7.106043,1129.718121,24092.115156,539.098015,...,352.316182,513.875834,151.541889,79966.872909,2503.574110,158.979905,4203.214837,1076.863253,1266.328571,1.0
2616,7.482791,2762.578281,61.148364,1392.498312,859.400762,68703.997690,8.171865,1144.683372,26067.249758,573.576256,...,369.190878,762.005391,186.093432,88433.731783,3016.978181,140.076140,3555.908171,1520.730490,1048.160521,1.0
2617,7.025504,2140.897416,51.118026,1443.042308,477.166794,62592.155832,7.276065,1076.379113,30385.770580,435.156713,...,304.732203,604.196979,205.400529,90870.332035,2217.251295,147.934237,3108.122040,1494.507584,1039.312624,1.0
2618,7.117579,2386.872036,45.460240,1325.290872,484.795746,62441.854412,6.387037,1134.107418,33062.346734,769.455836,...,335.348881,806.878019,186.199680,70863.551287,2141.885778,177.563928,2708.795736,1189.264287,1263.825228,1.0


The values have not yet been scaled.

# Analytical Question:

With the given dataset, can a model be built that can predict the potibility of water reliably and with good recall?

# Model Selection and training

First I'll test a plethora of models to see which one performs the best using cross validation. I'll use the average of the accuracy, recall, precision and f1 score to determine the best model.

In [6]:
models = [
    # Linear Models
    ('Logistic Regression', LogisticRegression(random_state=42, max_iter=1000)),
    
    # Tree Models
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),
    
    # Support Vector Machines
    ('SVM RBF', SVC(kernel='rbf', random_state=42)),
    ('Linear SVC', LinearSVC(random_state=42, max_iter=1000)),
    
    # Neighbors
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    
    # Neural Networks
    ('Neural Network', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)),
    
    # Naive Bayes
    ('Gaussian NB', GaussianNB()),
    
    # Ensemble Methods
    ('AdaBoost', AdaBoostClassifier(random_state=42)),
    ('Extra Trees', ExtraTreesClassifier(random_state=42)),
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),
]

model_names = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

for name, model in models:
    print(f'Running {name}')
    pipeline = make_pipeline(StandardScaler(), model)
    predictions = cross_val_predict(pipeline, train_df.drop(columns='not_potible'), train_df['not_potible'], cv=5)
    
    # Calculate metrics
    accuracy = accuracy_score(train_df['not_potible'], predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(train_df['not_potible'], predictions, average='binary')
    
    # Append results to lists
    model_names.append(name)
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Create DataFrame
results_df = pd.DataFrame({
    'Accuracy': accuracies,
    'Precision': precisions,
    'Recall': recalls,
    'F1 Score': f1_scores
}, index=model_names)
results_df

Running Logistic Regression
Running Decision Tree
Running SVM RBF
Running Linear SVC
Running KNN
Running Neural Network
Running Gaussian NB
Running AdaBoost
Running Extra Trees
Running Random Forest
Running Gradient Boosting


,Accuracy,Precision,Recall,F1 Score
Logistic Regression,0.649237,0.646235,0.930057,0.762594
Decision Tree,0.603435,0.671895,0.674858,0.673373
SVM RBF,0.673282,0.669291,0.910523,0.771490
Linear SVC,0.649237,0.647657,0.923125,0.761237
KNN,0.639695,0.674633,0.782609,0.724621
Neural Network,0.637023,0.695332,0.713296,0.704199
Gaussian NB,0.615267,0.649458,0.792691,0.713961
AdaBoost,0.625191,0.630444,0.921235,0.748592
Extra Trees,0.662977,0.680698,0.835539,0.750212
Random Forest,0.662595,0.685684,0.817895,0.745977


In [7]:
results_df['combined'] = results_df.mean(axis=1)
results_df.sort_values('combined', ascending=False)

,Accuracy,Precision,Recall,F1 Score,combined
SVM RBF,0.673282,0.669291,0.910523,0.771490,0.756147
Logistic Regression,0.649237,0.646235,0.930057,0.762594,0.747030
Linear SVC,0.649237,0.647657,0.923125,0.761237,0.745314
Gradient Boosting,0.655725,0.666667,0.863264,0.752334,0.734497
Extra Trees,0.662977,0.680698,0.835539,0.750212,0.732357
AdaBoost,0.625191,0.630444,0.921235,0.748592,0.731365
Random Forest,0.662595,0.685684,0.817895,0.745977,0.728038
KNN,0.639695,0.674633,0.782609,0.724621,0.705389
Gaussian NB,0.615267,0.649458,0.792691,0.713961,0.692844
Neural Network,0.637023,0.695332,0.713296,0.704199,0.687462


SVM with an RBF kernel did best.

I want to try tuning hyperparameters for the top 3 models to see if I can get better performance. I'll skip linear SVC because if I do I'll test a SVM, linear model, and an ensemble model.

## SVM

I'll use the F1 score as the scoring metric for the hyperparameter tuning.

In [8]:
svm_pipe = make_pipeline(StandardScaler(), SVC(kernel='rbf', random_state=42))

svm_params = {
    'svc__C': np.logspace(-3, 3, 7),
    'svc__gamma': np.logspace(-3, 3, 7)
}

svm_search = GridSearchCV(svm_pipe, svm_params, cv=5, scoring='f1', n_jobs=-1, verbose=1)
svm_search.fit(train_df.drop(columns='not_potible'), train_df['not_potible'])

Fitting 5 folds for each of 49 candidates, totalling 245 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC(random_state=42))]),
             n_jobs=-1,
             param_grid={'svc__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'svc__gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
             scoring='f1', verbose=1)

In [9]:
svm_search.best_params_, svm_search.best_score_

({'svc__C': np.float64(1000.0), 'svc__gamma': np.float64(0.001)},
 np.float64(0.7715106927110401))

That's slightly better than the default model.

## Logistic Regression

In [10]:
lr_pipe = make_pipeline(StandardScaler(), LogisticRegression(random_state=42, max_iter=1000))

lr_params = {
    'logisticregression__C': np.logspace(-3, 3, 7)
}

lr_search = GridSearchCV(lr_pipe, lr_params, cv=5, scoring='f1', n_jobs=-1, verbose=1)
lr_search.fit(train_df.drop(columns='not_potible'), train_df['not_potible'])

Fitting 5 folds for each of 7 candidates, totalling 35 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('logisticregression',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=42))]),
             n_jobs=-1,
             param_grid={'logisticregression__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
             scoring='f1', verbose=1)

In [11]:
lr_search.best_params_, lr_search.best_score_

({'logisticregression__C': np.float64(100.0)}, np.float64(0.7644273607094355))

Slightly better than the base logistic regression model, but not as good as the SVM.

## Gradient Boosting

In [12]:
gb_pipe = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=42, n_estimators=256))

gb_params = {
    'gradientboostingclassifier__learning_rate': np.logspace(-3, 0, 4),
    'gradientboostingclassifier__max_depth': [3, 5, 7]
}

gb_search = GridSearchCV(gb_pipe, gb_params, cv=5, scoring='f1', n_jobs=-1, verbose=1)
gb_search.fit(train_df.drop(columns='not_potible'), train_df['not_potible'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('gradientboostingclassifier',
                                        GradientBoostingClassifier(n_estimators=256,
                                                                   random_state=42))]),
             n_jobs=-1,
             param_grid={'gradientboostingclassifier__learning_rate': array([0.001, 0.01 , 0.1  , 1.   ]),
                         'gradientboostingclassifier__max_depth': [3, 5, 7]},
             scoring='f1', verbose=1)

In [13]:
gb_search.best_params_, gb_search.best_score_

({'gradientboostingclassifier__learning_rate': np.float64(0.01),
  'gradientboostingclassifier__max_depth': 3},
 np.float64(0.7633801552149728))

Again, better than the base model, but not as good as the SVM.

## Ensemble

Next I'll try a voting ensemble of the top 3 models.

In [14]:
scaler = StandardScaler()

svm = SVC(C=1000.0, gamma=0.001, kernel='rbf', probability=True, random_state=42)
lr = LogisticRegression(C=100.0, random_state=42, max_iter=1000)
gb = GradientBoostingClassifier(learning_rate=0.01, max_depth=3, n_estimators=256, random_state=42)

voting = VotingClassifier([
    ('SVM', svm),
    ('LR', lr),
    ('GB', gb)
], voting='soft')

voting_pipe = make_pipeline(scaler, voting)

voting_predictions = cross_val_predict(voting_pipe, train_df.drop(columns='not_potible'), train_df['not_potible'], cv=5)

accuracy_score(train_df['not_potible'], voting_predictions), precision_recall_fscore_support(train_df['not_potible'], voting_predictions, average='binary')

(0.6603053435114504,
 (0.6557890031291909, 0.9243856332703214, 0.7672594142259415, None))

In [15]:
results_df.loc['Voting'] = [accuracy_score(train_df['not_potible'], voting_predictions), *precision_recall_fscore_support(train_df['not_potible'], voting_predictions, average='binary')[:3], np.nan]
results_df.loc['Voting', 'combined'] = results_df.loc['Voting'].mean()
results_df.sort_values('combined', ascending=False)

,Accuracy,Precision,Recall,F1 Score,combined
SVM RBF,0.673282,0.669291,0.910523,0.771490,0.756147
Voting,0.660305,0.655789,0.924386,0.767259,0.751935
Logistic Regression,0.649237,0.646235,0.930057,0.762594,0.747030
Linear SVC,0.649237,0.647657,0.923125,0.761237,0.745314
Gradient Boosting,0.655725,0.666667,0.863264,0.752334,0.734497
Extra Trees,0.662977,0.680698,0.835539,0.750212,0.732357
AdaBoost,0.625191,0.630444,0.921235,0.748592,0.731365
Random Forest,0.662595,0.685684,0.817895,0.745977,0.728038
KNN,0.639695,0.674633,0.782609,0.724621,0.705389
Gaussian NB,0.615267,0.649458,0.792691,0.713961,0.692844


Well, in the end it seems that the SVM model is the best one.

## Feature selection

First, I'll find the best hyperparameters for the SVM model by doing a more exhaustive search. Then I'll do forward sequential selection to produce a more compact and hopefully better model.

In [16]:
svm_params = {
    'svc__C': np.logspace(-3, 3, 10),
    'svc__gamma': np.logspace(-3, 3, 8).tolist() + ['scale', 'auto']
}

svm_search = GridSearchCV(svm_pipe, svm_params, cv=5, scoring='f1', n_jobs=-1, verbose=1)
svm_search.fit(train_df.drop(columns='not_potible'), train_df['not_potible'])

Fitting 5 folds for each of 100 candidates, totalling 500 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC(random_state=42))]),
             n_jobs=-1,
             param_grid={'svc__C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                         'svc__gamma': [0.001, 0.0071968567300115215,
                                        0.0517947467923121, 0.3727593720314938,
                                        2.6826957952797246, 19.306977288832496,
                                        138.9495494373136, 1000.0, 'scale',
                                        'auto']},
             scoring='f1', verbose=1)

In [17]:
svm_search.best_params_, svm_search.best_score_

({'svc__C': np.float64(0.46415888336127775), 'svc__gamma': 'scale'},
 np.float64(0.7750481033490288))

In [18]:
svm_pipe.set_params(**svm_search.best_params_)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=np.float64(0.46415888336127775), random_state=42))])

In [19]:
sfs = SequentialFeatureSelector(svm_pipe, tol=0.001, scoring='f1', n_jobs=-1, cv=5)
sfs.fit(train_df.drop(columns='not_potible'), train_df['not_potible'])
sfs.get_feature_names_out()

array(['ph Sulfate', 'Hardness Solids_sqrt', 'Sulfate', 'Hardness',
       'Hardness Chloramines'], dtype=object)

These are the final features that will be used in the final model. 3 are engineered features and 2 are original features.

# Final Pipeline

In week 3 I set aside a raw test set to get a realistic recreation of new data. To use the data I need to make a front to back pipeline that will preprocess the data and then make predictions using the final model.

Going through all the previous notbooks, this function will do all the preprocessing steps that the training data in this notebook went through.

In [28]:
def preprocess_transform(df):
    df = df.copy()
    # Solids sqrt
    df['Solids_sqrt'] = df.Solids**0.5
    df.drop(columns='Solids', inplace=True)

    # Mean impute
    imputer = SimpleImputer(strategy='mean')
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    # Feature engineering
    df['ph Sulfate'] = df.ph * df.Sulfate
    df['Hardness Solids_sqrt'] = df.Hardness * df.Solids_sqrt
    df['Hardness Chloramines'] = df.Hardness * df.Chloramines

    # Feature selection
    df = df[['ph Sulfate', 'Hardness Solids_sqrt', 'Sulfate', 'Hardness', 'Hardness Chloramines']]

    return df

This will create the final pipeline that can go from raw data to predictions.

In [55]:
preprocess = FunctionTransformer(preprocess_transform)

final_pipe = make_pipeline(preprocess, svm_pipe)
final_pipe.named_steps

{'functiontransformer': FunctionTransformer(func=<function preprocess_transform at 0x7b27d5413a60>),
 'pipeline': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('svc',
                  SVC(C=np.float64(0.46415888336127775), random_state=42))])}

I had to go back and save the raw training data in the raw schema because I only saved the test set previously. I'll load that data and use it to train the final model.

In [56]:
raw_train_df = pd.read_sql('SELECT * FROM raw.train_set', db_con)
raw_train_df['Potability'] = 1 - raw_train_df.Potability

raw_preds = cross_val_predict(final_pipe, raw_train_df.drop(columns=['Potability']), raw_train_df['Potability'], cv=5)

In [57]:
accuracy_score(raw_train_df['Potability'], raw_preds), precision_recall_fscore_support(raw_train_df['Potability'], raw_preds, average='binary')

(0.6759541984732824,
 (0.6617002629272568, 0.9514807813484563, 0.7805634530886534, None))

Just to make sure this pipeline works properly with the raw data, here is the metrics for the final model using the raw training data. The metrics are very similar to the cross validation metrics, so I'm confident that this pipeline is working properly.

Fitting to the full raw training data.

In [67]:
final_pipe.set_params(pipeline__svc__probability=True)
final_pipe.fit(raw_train_df.drop(columns=['Potability']), raw_train_df['Potability'])

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function preprocess_transform at 0x7b27d5413a60>)),
                ('pipeline',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('svc',
                                  SVC(C=np.float64(0.46415888336127775),
                                      probability=True, random_state=42))]))])

# Save final pipeline

In [68]:
joblib.dump((final_pipe, preprocess_transform), 'final_pipe.joblib')

['final_pipe.joblib']